In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
text=open("stocks_txt.txt").read().lower()
len(text)

In [ ]:
# wordcloud creation
text=open("stocks_txt.txt", 'r', encoding='utf-8')
wordDict = {}
while True:
    line = text.readline()
    if not line:
        break
    line= line.replace(',',' ')
    line = line.replace('.',' ')
    line= line.replace("'",' ')
    line= line.replace('"',' ')
    line = line.replace(':',' ')
    line = line.replace(';', ' ')
    line = line.replace('–','  ')
    line = line.replace('(','  ')
    line = line.replace('“','  ')
    line = line.replace('\n','  ')
    line = line.replace(')','  ')
    
    wordlist = line.split(' ')
    
    for w in wordlist:
        if len(w)>4:
            if w in wordDict:
                wordDict[w]+=1
            else:
                wordDict[w]=1
    
text.close()

In [ ]:
wordDict

In [ ]:
from wordcloud import WordCloud

In [ ]:
wordcloud = WordCloud(background_color='white',\
                      colormap='Dark2',\
                      max_words=500,
                      width=500,\
                      height=300,\
                      contour_width=2, contour_color='steelblue',
                     collocations=True).generate_from_frequencies(wordDict)

In [ ]:
plt.figure(figsize=(14,6))
plt.axis('off')
plt.imshow(wordcloud)

In [ ]:
text=open("stocks_txt.txt").read().lower()

In [ ]:
maxlen=70
sentences=[]
next_chars=[]

In [ ]:
#prepare the dataset of input to output pairs encoded as integers
for i in range(len(text)-maxlen):
    #selecting from the text the first to 70th letters and appending to sentence array
    sentences.append(text[i:i+maxlen])
    # selecting each 70th letter and appending them in the next chars array
    next_chars.append(text[i+maxlen])

In [ ]:
len(sentences)

In [ ]:
len(next_chars)

In [18]:
for i in range(80):
    print(sentences[i],'-->',next_chars[i])
    print('--------------------')

top stocks to watch today.
as improving pandemic conditions continue t --> o
--------------------
op stocks to watch today.
as improving pandemic conditions continue to -->  
--------------------
p stocks to watch today.
as improving pandemic conditions continue to  --> s
--------------------
 stocks to watch today.
as improving pandemic conditions continue to s --> e
--------------------
stocks to watch today.
as improving pandemic conditions continue to se --> t
--------------------
tocks to watch today.
as improving pandemic conditions continue to set -->  
--------------------
ocks to watch today.
as improving pandemic conditions continue to set  --> t
--------------------
cks to watch today.
as improving pandemic conditions continue to set t --> h
--------------------
ks to watch today.
as improving pandemic conditions continue to set th --> e
--------------------
s to watch today.
as improving pandemic conditions continue to set the -->  
--------------------
 to watch today.
as 

In [19]:
# converts the texts into letters
# turns them into sorted listed format
chars=sorted(list(set(text)))
print('unique chars:', len(chars))

unique chars: 63


In [20]:
print(chars, end=' ')

['\n', ' ', '"', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '·', 'í', 'ñ', '—', '’', '“', '”', '…'] 

In [21]:
# create a new dictionary
char_indices={}
for char in chars:
    char_indices[char]=chars.index(char)
    for i, j in char_indices.items():
        char_indices[i] = int(j)

In [22]:
char_indices

{'\n': 0,
 ' ': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 '+': 9,
 ',': 10,
 '-': 11,
 '.': 12,
 '/': 13,
 '0': 14,
 '1': 15,
 '2': 16,
 '3': 17,
 '4': 18,
 '5': 19,
 '6': 20,
 '7': 21,
 '8': 22,
 '9': 23,
 ':': 24,
 ';': 25,
 '?': 26,
 '[': 27,
 ']': 28,
 'a': 29,
 'b': 30,
 'c': 31,
 'd': 32,
 'e': 33,
 'f': 34,
 'g': 35,
 'h': 36,
 'i': 37,
 'j': 38,
 'k': 39,
 'l': 40,
 'm': 41,
 'n': 42,
 'o': 43,
 'p': 44,
 'q': 45,
 'r': 46,
 's': 47,
 't': 48,
 'u': 49,
 'v': 50,
 'w': 51,
 'x': 52,
 'y': 53,
 'z': 54,
 '·': 55,
 'í': 56,
 'ñ': 57,
 '—': 58,
 '’': 59,
 '“': 60,
 '”': 61,
 '…': 62}

In [23]:
# vectorizing the sentences
array_x = (len(sentences), maxlen, len(chars))
array_y = (len(sentences), len(chars))
#converting the truncated matrixes into boolean value
X=np.zeros(array_x, dtype=np.bool)
y=np.zeros(array_y, dtype=np.bool)

In [24]:
# lets get each letter of the sentences
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]]=1
    y[i, char_indices[next_chars[i]]]=1

In [ ]:
model=keras.Sequential([
    keras.layers.LSTM(128, input_shape=(maxlen, len(chars))),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(len(chars), activation='softmax')])

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
def maxchar(preds):
    preds=np.asarray(preds).astype('float64')
    #getting the max-arguement of the predicted
    return np.argmax(preds)

In [ ]:
def sample(preds, index):
    preds=np.asarray(preds).astype('float64')
    preds=np.log(preds)/index
    exp_preds=np.exp(preds)
    preds=exp_preds/np.sum(exp_preds)
    probas=np.random.multinomial(1,preds)
    return np.argmax(probas)

In [ ]:
# select randomly
start = np.random.randint(0, len(sentences)-1)

In [ ]:
seed_text = text[start: start+maxlen]
print(seed_text)

In [ ]:
for epoch in range(1,30):
    model.fit(X,y,batch_size=128, epochs=1)
    generated_text=seed_text
    print(generated_text)

    for i in range(800):
        sampled=np.zeros((1,maxlen, len(chars)))
        for t,char in enumerate(generated_text):
            sampled[0,t,char_indices[char]]=1.

        preds=model.predict(sampled, verbose=0)[0]
        next_index=sample(preds, 0.5)
        next_char=chars[next_index]

        generated_text+=next_char
        generated_text=generated_text[1:]

        print(next_char, end='', flush=True)
        
    print()

917/917 [==============================] - 181s 197ms/step - loss: 1.6411
at it had bought a stake in la fete du rose, a premium rose wine label
t, the sect to the stocks to invest in 2021. is a stock and realth and maring that the company operate stocks to invest in 2021. its stocks to buy industry seal held and has ale stock stocks to invest in 2021. and the procedstors digetrong to invest in 2021. in the parced bething company that market in the past couther to be the dist of the company tast bate stocks al a perlice to for the company insted that of these company ecreven in the firm astall to net company as alitas experest and beat a longed to a be a stock to ton to invest in 2021.

by the rese than its stares to be the beto stocks to invest in 2021-best continue to invest yor the stock and a stock to invest in 2021. it is real year of the company can and continue to deesing shares in the firm ealland and the recent of the comp
917/917 [==============================] - 203s 221ms/

 to believe promite the wergh in over 20 mere than 12% now. investors of the sect of 2021. the company bolect the company with a reat income and nom. in the stock in minerow. aperating is a renuration by the company has been the part to feel the first quarter on our list of 10 best wine stocks to invest in 2021. the porfor that the company as a company that was and the year of the company was been to out exciting to the best stocks to invest in 2021. in at the company as a sales to industry improve stocks that the company are to for adverted in march 2017. in shopping and the company and the firm has also be for the company and an start that its firm hew ispers market and comenting the begines to fut the company a solar inc. (nasdaq: nvd) is a comente market stocks to your now the company 
917/917 [==============================] - 3084s 3s/step - loss: 1.3688
at it had bought a stake in la fete du rose, a premium rose wine label
t to revenue section in the enditients bate stock to pay

In [226]:
# for i in range(len(t)-ma):
#     s.append(t[i:i+ma])
#     d.append(t[i+ma])

In [227]:
# d

[' ', 'n', 'a', 'm', 'e', ' ', 'i', 's', ' ', 'm', 'i', 'k', 'e']

In [228]:
s

['my', 'y ', ' n', 'na', 'am', 'me', 'e ', ' i', 'is', 's ', ' m', 'mi', 'ik']

In [208]:
# ch=sorted(list(set(t)))

In [209]:
# ch

[' ', 'a', 'e', 'i', 'k', 'm', 'n', 's', 'y']

In [100]:
# ch=set(t)

In [120]:
# ch

[' ', 'a', 'e', 'i', 'k', 'm', 'n', 's', 'y']

In [210]:
# ch_ind=dict((c, ch.index(c)) for c in ch)

In [211]:
# ch_ind

{' ': 0, 'a': 1, 'e': 2, 'i': 3, 'k': 4, 'm': 5, 'n': 6, 's': 7, 'y': 8}

In [ ]:
# 'my name is mike'

In [143]:
# ch_ind = {}

In [144]:
for i in ch:
    ch_ind[i]=ch.index(i)

In [145]:
ch_ind

{' ': 0, 'a': 1, 'e': 2, 'i': 3, 'k': 4, 'm': 5, 'n': 6, 's': 7, 'y': 8}

In [212]:
le =np.zeros((len(s), ma, len(ch)), dtype=np.bool)

In [213]:
le

array([[[False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False]],

       [[False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False]],

       [[False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False]],

       [[False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False]],

       [[False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False]],

       [[False, False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False]],

       [[False, False, False, False, False, False, False, False, False],
        [False, False, False, False, Fa

In [214]:
e = np.zeros((len(s), ma), dtype=np.bool)

In [215]:
e

array([[False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False],
       [False, False]])

In [156]:
f = (2,2,3)

In [158]:
np.zeros(f, dtype=np.bool)

array([[[False, False, False],
        [False, False, False]],

       [[False, False, False],
        [False, False, False]]])

In [163]:
j = [[[1,2], [2,3], [1,3],[4,1]]]

In [166]:
for i in j:
    print(i)

[[1, 2], [2, 3], [1, 3], [4, 1]]


In [238]:
for i, e in enumerate(s):
    for t, c in enumerate(e):
         le[i, t, ch_ind[ch]]=1

TypeError: unhashable type: 'list'

In [236]:
q = ['ghee','gje2','gh']

In [237]:
for i, j in enumerate(q):
    print(i)
        

0
1
2


In [273]:
str_i = 5

In [280]:
t[str_i: str_i+ma]

'me'